In [1]:
ROOT_DATA = "input_a3"
TRANSFER_LEARNING_DATA = f"{ROOT_DATA}/transfer_learning"
CHARACTER_DATA = f"{ROOT_DATA}/characters"

TL_INPUT_DATA = f"{ROOT_DATA}/tl_input"
TL_OUTPUT_DATA = f"{ROOT_DATA}/tl_output"
TL_COMBINED_DATA = f"{ROOT_DATA}/tl_combined"

PIX2PIX_INPUT_DATA = f"{ROOT_DATA}/pix2pix_input"
PIX2PIX_OUTPUT_DATA = f"{ROOT_DATA}/pix2pix_output"
PIX2PIX_COMBINED_DATA = f"{ROOT_DATA}/pix2pix_combined"

In [2]:
# get list of the files in transfer learning
import os, glob, random
import common_functions as core

random.seed(187)

transfer_learn_images = sorted(list(glob.glob(TRANSFER_LEARNING_DATA + "/*.png")))
character_images = sorted(list(glob.glob(CHARACTER_DATA + "/*.png")))

random.shuffle(transfer_learn_images)
random.shuffle(character_images)

split_folders = ("train", "test", "val")

In [3]:
def prepare_data(input_images, model_inp_dir, model_out_dir, split = (0.8, 0.9, 1.0)):
    counter = 0
    train_end = int(len(input_images) * split[0])
    test_end = int(len(input_images) * split[1])
    sketch_per_img = 3
    for image in input_images:
        magenta_bg_image = core.pil_to_opencv(core.load_64x64_with_magenta_bg(image))
        for sketch in range(1, sketch_per_img + 1):
            sketch_image = core.create_sketch(magenta_bg_image, levels=sketch, magenta_bg=True)
            filename = f"{counter:05}.png"
            current_split_folder = split_folders[0]
            if train_end < counter < test_end:
                current_split_folder = split_folders[1]
            elif counter >= test_end:
                current_split_folder = split_folders[2]
            # ----------- Write to model_input and output directories -----------
            model_input_path = os.path.join(model_inp_dir, current_split_folder)
            os.makedirs(model_input_path, exist_ok=True)
            core.write_image(sketch_image, os.path.join(model_input_path, filename))
            
            model_out_path = os.path.join(model_out_dir, current_split_folder)
            os.makedirs(model_out_path, exist_ok=True)
            core.write_image(magenta_bg_image, os.path.join(model_out_path, filename))
            
            counter += 1

In [4]:
prepare_data(transfer_learn_images, TL_INPUT_DATA, TL_OUTPUT_DATA)

In [5]:
prepare_data(character_images, PIX2PIX_INPUT_DATA, PIX2PIX_OUTPUT_DATA)

In [6]:
os.makedirs(TL_COMBINED_DATA, exist_ok=True)
!python ./pix2pix/datasets/combine_A_and_B.py --fold_A ./input_a3/tl_input --fold_B ./input_a3/tl_output --fold_AB ./input_a3/tl_combined --no_multiprocessing

[fold_A] =  ./input_a3/tl_input
[fold_B] =  ./input_a3/tl_output
[fold_AB] =  ./input_a3/tl_combined
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 338/338 images
split = test, number of images = 338
split = train, use 2709/2709 images
split = train, number of images = 2709
split = val, use 7111/7111 images
split = val, number of images = 7111


In [7]:
os.makedirs(PIX2PIX_COMBINED_DATA, exist_ok=True)
!python ./pix2pix/datasets/combine_A_and_B.py --fold_A ./input_a3/pix2pix_input --fold_B ./input_a3/pix2pix_output --fold_AB ./input_a3/pix2pix_combined --no_multiprocessing 

[fold_A] =  ./input_a3/pix2pix_input
[fold_B] =  ./input_a3/pix2pix_output
[fold_AB] =  ./input_a3/pix2pix_combined
[num_imgs] =  1000000
[use_AB] =  False
[no_multiprocessing] =  True
split = test, use 90/90 images
split = test, number of images = 90
split = train, use 730/730 images
split = train, number of images = 730
split = val, use 1916/1916 images
split = val, number of images = 1916
